In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import warnings
from datetime import datetime

# 불필요한 경고 메시지를 숨깁니다.
warnings.filterwarnings('ignore')

In [2]:
def predict(lotto_history_data, n_estimators=100, random_state=350, verbose=0):
    # 데이터를 Pandas DataFrame으로 변환합니다.
    df = pd.DataFrame(lotto_history_data, columns=['회차', '번호1', '번호2', '번호3', '번호4', '번호5', '번호6'])
    df = df.sort_values(by='회차').reset_index(drop=True)

    if verbose > 0:
        print("--- 로또 당첨 번호 이력 ---")
        print(df)

    # 기계 학습을 위한 데이터 준비
    # 각 회차의 당첨 번호(X)와 바로 다음 회차의 당첨 번호(y)를 사용합니다.
    X = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]
    y = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].shift(-1)

    # 마지막 행은 다음 회차 데이터가 없으므로 제거합니다.
    X = X[:-1]
    y = y.dropna()
    
    if verbose > 0:
        print("\n--- 훈련 데이터 (X) ---")
        print(X.head())
        print("\n--- 정답 데이터 (y) ---")
        print(y.head())

    # 머신러닝 모델 선택 및 훈련
    # 랜덤 포레스트 회귀 모델을 사용합니다.
    # the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
    # the n_estimators parameter specifies the number of decision trees in the forest. 
    # model = RandomForestRegressor(n_estimators=50, random_state=1000) # n_estimators: 만들 트리의 개수 => 3개 일치
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state) # n_estimators: 만들 트리의 개수
    model.fit(X, y)

    # 예측할 회차의 이전 회차 데이터 (1173회차)
    last_draw = df.iloc[-1][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.reshape(1, -1)

    if verbose > 0:
        print(f"\n--- {df.iloc[-1]['회차']}회차 데이터로 다음 회차 예측 ---")
        print(last_draw)

    # 다음 회차(1174회) 번호 예측
    predicted_numbers_set = []
    trial = 5
    for i in range(trial):
        predicted_numbers_float = model.predict(last_draw)

        # 예측된 번호 처리
        # 1. 소수점을 반올림하여 정수로 만듭니다.
        # 2. 1~45 사이의 값으로 보정합니다.
        # 3. 중복된 번호를 제거하고 6개를 선택합니다.
        predicted_numbers = set()
        for num in predicted_numbers_float[0]:
            # 반올림하여 정수로 변환
            int_num = int(round(num))
    
            # 1보다 작으면 1로, 45보다 크면 45로 보정
            if int_num < 1:
                int_num = 1
            elif int_num > 45:
                int_num = 45
            predicted_numbers.add(int_num)

        # 중복 제거 후 6개가 안되면, 부족한 만큼 다른 번호로 채웁니다.
        # (여기서는 가장 빈도가 높은 번호들 중 예측되지 않은 번호를 추가하는 방식을 사용)
        if len(predicted_numbers) < 6:
            all_numbers = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
            counts = pd.Series(all_numbers).value_counts()
    
            extra_needed = 6 - len(predicted_numbers)
            for num in counts.index:
                if extra_needed == 0:
                    break
                if num not in predicted_numbers:
                    predicted_numbers.add(num)
                    extra_needed -= 1
        # 최종 예측 번호를 정렬하여 출력
        final_prediction = sorted(list(predicted_numbers))[:6]
        predicted_numbers_set.append(final_prediction)
    return predicted_numbers_set


def test_prediction(round_nums,
                    actual_numbers,
                    bonus,
                    predicted_numbers_set,
                    n_estimators=500,
                    random_state=500,
                    only_one=True,
                    verbose=0):
    if verbose>0:
        print("======================================")
        print(f"\n--- 참고: {round_nums}회 실제 당첨 번호 ---")
        print(f"당첨번호: {actual_numbers}, 보너스: {bonus}")

    matched_len = 0

    for final_prediction in predicted_numbers_set:
        # 예측 결과와 실제 결과 비교
        matching_numbers = set(final_prediction).intersection(set(actual_numbers))
        if verbose > 0:
            print(f"\n--- 예측과 실제 결과 비교 ---")
            print(f"일치하는 번호: {sorted(list(matching_numbers))}")
            print(f"일치 개수: {len(matching_numbers)}")
            print(F"예측 번호: {final_prediction}")
            print(f"n_estimators: {n_estimators}")
            print(f"random_state: {random_state}")
        matched_len = len(matching_numbers)
        if only_one:
            break
    return matched_len

In [5]:
# 실제 1174회 당첨 번호
actual_numbers = [
    [1, 2, 3, 4, 5, 6],
    [8, 11, 14, 17, 36, 39],
    [1, 5, 18, 20, 30, 35],
    [7, 9, 24, 40, 42, 44],
]
bonus = [0, 22, 22]
round_nums = [1175, 1174, 1173]
# 역대 로또 당첨 번호 데이터 (최신 회차부터 순서대로)
# 실제로는 더 많은 데이터가 필요하지만, 예시를 위해 일부만 사용합니다.
# 데이터 출처: https://data.soledot.com/lottowinnumber/fo/lottowinnumberlist.sd
lotto_history_data = [
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
    [1166, 14, 23, 25, 27, 29, 42],
    [1165, 6, 7, 27, 29, 38, 45],
    [1164, 17, 18, 23, 25, 38, 39],
    [1163, 2, 13, 15, 16, 33, 43],
    [1162, 20, 21, 22, 25, 28, 29],
    [1161, 2, 12, 20, 24, 34, 42],
    [1160, 7, 13, 18, 36, 39, 45],
    [1159, 3, 9, 27, 28, 38, 39],
    [1158, 21, 25, 27, 32, 37,38],
    [1157, 5, 7, 12, 20, 25, 26],
    [1156, 30, 31, 34, 39, 41, 45],
    [1155, 10, 16, 19, 27, 37, 38],
    [1154, 4, 8, 22, 26, 32, 38],
    [1153, 1, 9, 10, 13, 35, 44],
    [1152, 30, 31, 32, 35, 36, 37],
    [1151, 2, 3, 9, 15, 27, 29],
    [1150, 8, 9, 18, 35, 39, 45],
]

r_pos = [
    (0, len(lotto_history_data)-3+1),
    (1, len(lotto_history_data)-2+1),
    (2, len(lotto_history_data)-1+1),
]

results_set=[]

**Notice**
- the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
- the n_estimators parameter specifies the number of decision trees in the forest. This parameter is crucial as it directly impacts the model's performance and computational cost.

In [6]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=8
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:16:32.401150]
proceed [now=2025-06-06 20:17:49.782536, 1000]
proceed [now=2025-06-06 20:19:07.718582, 2000]
proceed [now=2025-06-06 20:20:25.659409, 3000]
completed [now=2025-06-06 20:20:25.659560]


In [7]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=7
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:20:56.937388]
proceed [now=2025-06-06 20:22:14.664391, 1000]
proceed [now=2025-06-06 20:23:32.075223, 2000]
proceed [now=2025-06-06 20:24:49.820968, 3000]
completed [now=2025-06-06 20:24:49.821138]


In [8]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=6
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:25:03.720069]
found =  45 307800 4 3 6
found =  45 350600 3 3 6
found =  45 390200 3 3 6
proceed [now=2025-06-06 20:26:22.472455, 1000]
proceed [now=2025-06-06 20:27:40.474009, 2000]
proceed [now=2025-06-06 20:29:00.172547, 3000]
completed [now=2025-06-06 20:29:00.172731]


In [9]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=5
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:30:20.242551]
found =  45 312700 3 3 5
found =  45 335100 3 3 5
found =  45 385400 3 3 5
proceed [now=2025-06-06 20:31:36.044152, 1000]
found =  45 492700 3 3 5
found =  45 494200 3 3 5
found =  45 496200 3 3 5
proceed [now=2025-06-06 20:32:50.550862, 2000]
found =  45 567900 3 3 5
found =  45 588500 3 3 5
proceed [now=2025-06-06 20:34:04.902577, 3000]
completed [now=2025-06-06 20:34:04.902738]


In [10]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=4
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:34:04.909414]
found =  45 381000 3 3 4
proceed [now=2025-06-06 20:35:20.316071, 1000]
proceed [now=2025-06-06 20:36:39.565802, 2000]
proceed [now=2025-06-06 20:38:03.219791, 3000]
completed [now=2025-06-06 20:38:03.219941]


In [11]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=3
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:38:09.088138]
found =  45 317700 3 4 3
proceed [now=2025-06-06 20:39:33.450991, 1000]
proceed [now=2025-06-06 20:40:55.397529, 2000]
proceed [now=2025-06-06 20:42:16.744740, 3000]
completed [now=2025-06-06 20:42:16.744909]


In [12]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=2
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:42:16.751616]
found =  45 368200 3 3 2
found =  45 394600 3 3 2
proceed [now=2025-06-06 20:43:39.432755, 1000]
proceed [now=2025-06-06 20:45:00.824109, 2000]
proceed [now=2025-06-06 20:46:21.872693, 3000]
completed [now=2025-06-06 20:46:21.872866]


In [13]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=1
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:46:21.879322]
found =  45 321800 3 3 1
proceed [now=2025-06-06 20:47:40.049569, 1000]
found =  45 492300 3 3 1
proceed [now=2025-06-06 20:49:01.121785, 2000]
proceed [now=2025-06-06 20:50:23.747103, 3000]
completed [now=2025-06-06 20:50:23.747280]


In [14]:
n_estimators=45
random_state=2000
begin_pos=300000
end_pos=600000
minus_value=0
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 2 and predict_len2 > 2:
        results_set.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 20:50:23.754587]
proceed [now=2025-06-06 20:51:45.744618, 1000]
proceed [now=2025-06-06 20:53:06.365071, 2000]
proceed [now=2025-06-06 20:54:23.517615, 3000]
completed [now=2025-06-06 20:54:23.517767]


In [15]:
for result in results_set:
    print(result)

(45, 307800, 4, 3, 6)
(45, 350600, 3, 3, 6)
(45, 390200, 3, 3, 6)
(45, 312700, 3, 3, 5)
(45, 335100, 3, 3, 5)
(45, 385400, 3, 3, 5)
(45, 492700, 3, 3, 5)
(45, 494200, 3, 3, 5)
(45, 496200, 3, 3, 5)
(45, 567900, 3, 3, 5)
(45, 588500, 3, 3, 5)
(45, 381000, 3, 3, 4)
(45, 317700, 3, 4, 3)
(45, 368200, 3, 3, 2)
(45, 394600, 3, 3, 2)
(45, 321800, 3, 3, 1)
(45, 492300, 3, 3, 1)


In [61]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[4]
    results=[]
    print(f'start [now={datetime.now()}]')
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose=1)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:53:49.764707]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수: 4
예측 번호: [14, 17, 23, 29, 36, 39]
n_estimators: 45
random_state: 208000

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 20, 30]
일치 개수: 3
예측 번호: [5, 13, 20, 24, 30, 37]
n_estimators: 45
random_state: 208000
start [now=2025-06-06 19:53:49.859536]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수: 4
예측 번호: [14, 17, 23, 30, 36, 39]
n_estimators: 45
random_state: 101700

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 20, 30]
일치 개수: 3
예측 번호: [5, 13, 20, 25, 30, 36]
n_estimators: 45
random_state: 101700
start [now=2025-06-06 19:53:50.016935]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수:

In [63]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
print(f'start [now={datetime.now()}]')
results=[]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[4]    
    data_pos=0
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    results.append(predicted_numbers_set[0])
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[1],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')
print('Predicted Numbers.')
for result in results:
    print(result)

start [now=2025-06-06 19:59:26.719459]

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14]
일치 개수: 1
예측 번호: [9, 14, 24, 29, 37, 41]
n_estimators: 45
random_state: 208000

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: []
일치 개수: 0
예측 번호: [15, 19, 25, 30, 37, 41]
n_estimators: 45
random_state: 101700

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14]
일치 개수: 1
예측 번호: [14, 18, 25, 30, 37, 41]
n_estimators: 45
random_state: 185400

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 36]
일치 개수: 2
예측 번호: [11, 16, 24, 29, 36, 41]
n_estimators: 45
random_state: 96800

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [17, 36]
일치 개수: 2
예측 번호: [13, 17, 25, 30, 36, 40]
n_estimators: 45
random_state: 58600
completed [now=2025-06-06 19:59:26.918482]
Predicted 

In [32]:
print(results)

[]


```
[
[45,208000,4,3,5],
[45,101700,4,3,2],
[45,101700,4,3,2],
[45,185400,4,3,2],
[45,96800,4,3,1],
    [45,58600,3,3,0],
]
```

In [128]:
n_estimators=10
random_state=5000
data_pos=1
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0
                               )
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)
data_pos=2
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0)
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)


--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [36]
일치 개수: 1
예측 번호: [18, 19, 22, 27, 34, 36]
n_estimators: 10
random_state: 5000

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [18]
일치 개수: 1
예측 번호: [10, 18, 24, 28, 32, 40]
n_estimators: 10
random_state: 5000


In [ ]:
[
    ([14, 17, 23, 27, 34, 36], [14, 17, 36], (45, 1000), (1160, 1173),
    ([13, 16, 23, 27, 34, 36], [36], (45, 1300), (1160, 1173)),
    ([11, 15, 21, 30, 36, 39], [11, 36, 39], (45, 1000), (1160, 1173),

]